Testing
===

In [1]:
import sys; sys.path.append('..')
import time
import json
from math import ceil, floor
from os.path import join as opj, dirname

import tqdm
import torch
import pandas as pd
from torch.utils.data.dataloader import DataLoader
from torch.autograd import Variable
import torchvision.transforms as T

from src.dataset import StatoilIcebergDataset
from src.network import Net
from src.settings import logger
from src.tensorboard_logger import Logger
from src.utils import mkdir_r

/home/rlan/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Define Const

In [2]:
test_data_path = '/home/rlan/datasets/statoil-iceberg/test.json'

BASE_DIR = '/home/rlan/projects/kaggle/kaggle-statoil-iceberg'
CHECKPOINTS_PATH = opj(BASE_DIR, 'checkpoints')
BATCH_SIZE = 256
model_id = '1514653493'
model_epoch = 164

model_path = opj(CHECKPOINTS_PATH, model_id, 'model_%d' % model_epoch)

## Transform

In [3]:
transform = T.Compose([T.ToTensor()])

## Dataset

In [4]:
dataset = StatoilIcebergDataset(test_data_path, transform=transform)
loader = DataLoader(dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=8)

## Load Model Checkpoint

In [5]:
checkpoint = torch.load(model_path)
net = Net(input_channel=2).cuda() if torch.cuda.is_available() else Net(input_channel=2)
net.load_state_dict(checkpoint)
net.eval()

Net(
  (conv1): Conv2d (2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d (32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv5): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv6): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=20736, out_features=120)
  (fc1_bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True)


## Test

In [6]:
answers = torch.Tensor().cuda()
for i_batch, sampled_batch in enumerate(loader):
    data, target = sampled_batch

    if torch.cuda.is_available():
        data, target = Variable(data).cuda(), Variable(target).cuda()
    else:
        data, target = Variable(data), Variable(target)

    pred = net(data)
    pred_softmax = torch.nn.functional.softmax(pred)
    answers = torch.cat((answers, pred_softmax[:, 1].data), dim=0)

if torch.cuda.is_available():
    answers = answers.cpu()
answers = answers.numpy()

/home/rlan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


## Construct submission with both predictions and IDs

In [7]:
with open(test_data_path, 'r') as f:
    test_data = json.load(f)

test_ids = [d['id'] for d in test_data]
submission = pd.DataFrame({'id': test_ids, 'is_iceberg': answers})

## Write submission file

In [8]:
submission.to_csv(opj(BASE_DIR, 'submission.csv'), index=False)